In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import squidpy as sq

/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


## ADT+RNA

In [3]:
path="../datasets/10x_human_lymph_node_D1/"
adata2=sc.read_h5ad(path+"adata_ADT.h5ad")
adata1=sc.read_h5ad(path+"adata_RNA.h5ad")

adata = sc.concat([adata1, adata2], axis=1, merge='same')
adata.obs['sample'] = 'Human_Lymph_Node'
adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
adata.uns['spatial'] = {s: {} for s in adata.obs['sample'].unique()}
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='grid', n_neighs=4, n_rings=1)

adata1.layers["counts"] = adata1.X.copy()

sc.pp.normalize_total(adata1, target_sum=1e6)
sc.pp.log1p(adata1)

import scvi
scvi.model.SCVI.setup_anndata(
    adata1, 
    layer="counts", 
)
 
model1 = scvi.model.SCVI(adata1)
model1.to_device("cuda:1")
model1.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata1.obsm['X_scVI'] = model1.get_latent_representation()

adata2.layers["counts"] = adata2.X.copy()
scvi.model.SCVI.setup_anndata(
    adata2, 
    layer="counts",
)

model2 = scvi.model.SCVI(adata2)
model2.to_device("cuda:1")
model2.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1,batch_size=)

adata2.obsm['X_scVI'] = model2.get_latent_representation()

import cellcharter as cc

adata.obsm['X_scVI'] = np.concatenate([adata1.obsm['X_scVI'], adata2.obsm['X_scVI']], axis=1)
cc.gr.aggregate_neighbors(adata, n_layers=4, use_rep='X_scVI')
adata.write_h5ad(path.replace("datasets","results")+"adata_cellcharter.h5ad")

/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high

Epoch 101/400:  25%|███████████▎                                 | 101/400 [00:53<02:38,  1.89it/s, v_num=1, train_loss_step=3.65e+3, train_loss_epoch=3.57e+3]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 3828.741. Signaling Trainer to stop.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Epoch 341/400:  85%|█████████████████████████████████████████████▏       | 341/400 [02:24<00:25,  2.36it/s, v_num=1, train_loss_step=280, train_loss_epoch=279]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 282.310. Signaling Trainer to stop.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 390.70it/s]


## ADT+RNA+ATAC

In [23]:
path="../datasets/simulated_data/"
adata2=sc.read_h5ad(path+"adata_ADT.h5ad")
adata1=sc.read_h5ad(path+"adata_RNA.h5ad")
adata3=sc.read_h5ad(path+"adata_ATAC.h5ad")

adata = sc.concat([adata1, adata2,adata3], axis=1, merge='same')
adata.obs['sample'] = 'Hsimulated_data'
adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
adata.uns['spatial'] = {s: {} for s in adata.obs['sample'].unique()}
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='grid', n_neighs=3, n_rings=1)

adata1.layers["counts"] = adata1.X.copy()

sc.pp.normalize_total(adata1, target_sum=1e6)
sc.pp.log1p(adata1)

import scvi
scvi.model.SCVI.setup_anndata(
    adata1, 
    layer="counts", 
)
 
model1 = scvi.model.SCVI(adata1)
model1.to_device("cuda:1")
model1.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata1.obsm['X_scVI'] = model1.get_latent_representation()

adata2.layers["counts"] = adata2.X.copy()
scvi.model.SCVI.setup_anndata(
    adata2, 
    layer="counts",
)

model2 = scvi.model.SCVI(adata2)
model2.to_device("cuda:1")
model2.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata2.obsm['X_scVI'] = model2.get_latent_representation()

adata3.layers["counts"] = adata3.X.copy()
scvi.model.SCVI.setup_anndata(
    adata3, 
    layer="counts",
)

model3 = scvi.model.SCVI(adata3)
model3.to_device("cuda:1")
model3.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata3.obsm['X_scVI'] = model3.get_latent_representation()

import cellcharter as cc

adata.obsm['X_scVI'] = np.concatenate([adata1.obsm['X_scVI'], adata2.obsm['X_scVI'], adata3.obsm['X_scVI']], axis=1)
cc.gr.aggregate_neighbors(adata, n_layers=4, use_rep='X_scVI')
print(adata.obsm['X_cellcharter'].shape)
adata.write_h5ad(path.replace("datasets","results")+"adata_cellcharter.h5ad")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Epoch 180/400:  45%|████████████████████▎                        | 180/400 [00:36<00:44,  4.95it/s, v_num=1, train_loss_step=1.19e+3, train_loss_epoch=1.21e+3]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 1324.480. Signaling Trainer to stop.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Epoch 400/400: 100%|█████████████████████████████████████████████████████| 400/400 [01:19<00:00,  5.29it/s, v_num=1, train_loss_step=379, train_loss_epoch=326]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█████████████████████████████████████████████████████| 400/400 [01:19<00:00,  5.02it/s, v_num=1, train_loss_step=379, train_loss_epoch=326]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Epoch 175/400:  44%|███████████████████▋                         | 175/400 [00:35<00:45,  4.98it/s, v_num=1, train_loss_step=1.62e+3, train_loss_epoch=1.54e+3]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 1585.607. Signaling Trainer to stop.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 910.34it/s]

(1296, 150)


## RNA+ATAC

In [ ]:
for path in ["../datasets/Mouse_Brain_E11_S1/","../datasets/Mouse_Brain_E13_S1/","../datasets/Mouse_Brain_E15_S1/","../datasets/Mouse_Brain_E18_S1/"]:

    adata1=sc.read_h5ad(path+"adata_RNA.h5ad")
    adata2=sc.read_h5ad(path+"adata_ATAC.h5ad")
    
    adata = sc.concat([adata1, adata2], axis=1, merge='same')
    adata.obs['sample'] = 'Hsimulated_data'
    adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
    adata.uns['spatial'] = {s: {} for s in adata.obs['sample'].unique()}
    sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='grid', n_neighs=3, n_rings=1)
    
    adata1.layers["counts"] = adata1.X.copy()
    
    sc.pp.normalize_total(adata1, target_sum=1e6)
    sc.pp.log1p(adata1)
    
    import scvi
    scvi.model.SCVI.setup_anndata(
        adata1, 
        layer="counts", 
    )
     
    model1 = scvi.model.SCVI(adata1)
    model1.to_device("cuda:1")
    model1.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)
    
    adata1.obsm['X_scVI'] = model1.get_latent_representation()
    
    adata2.layers["counts"] = adata2.X.copy()
    scvi.model.SCVI.setup_anndata(
        adata2, 
        layer="counts",
    )
    
    model2 = scvi.model.SCVI(adata2)
    model2.to_device("cuda:1")
    model2.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)
    
    adata2.obsm['X_scVI'] = model2.get_latent_representation()
    

    import cellcharter as cc
    
    adata.obsm['X_scVI'] = np.concatenate([adata1.obsm['X_scVI'], adata2.obsm['X_scVI']], axis=1)
    cc.gr.aggregate_neighbors(adata, n_layers=4, use_rep='X_scVI')
    print(adata.obsm['X_cellcharter'].shape)
    adata.write_h5ad(path.replace("datasets","results")+"adata_cellcharter.h5ad")

In [2]:
%reload_ext memory_profiler

path='../datasets/Mouse_Brain_H3K27me3/'

!nvidia-smi
adata1=sc.read_h5ad(path+"adata_RNA.h5ad")
adata2=sc.read_h5ad(path+"adata_ATAC.h5ad")

adata = sc.concat([adata1, adata2], axis=1, merge='same')
adata.obs['sample'] = 'Hsimulated_data'
adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
adata.uns['spatial'] = {s: {} for s in adata.obs['sample'].unique()}
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='grid', n_neighs=3, n_rings=1)

adata1.layers["counts"] = adata1.X.copy()

sc.pp.normalize_total(adata1, target_sum=1e6)
sc.pp.log1p(adata1)

import scvi
scvi.model.SCVI.setup_anndata(
    adata1, 
    layer="counts", 
)
 
model1 = scvi.model.SCVI(adata1)
model1.to_device("cuda:1")
%time %memit model1.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata1.obsm['X_scVI'] = model1.get_latent_representation()

adata2.layers["counts"] = adata2.X.copy()
scvi.model.SCVI.setup_anndata(
    adata2, 
    layer="counts",
)

model2 = scvi.model.SCVI(adata2)
model2.to_device("cuda:1")
%time %memit model2.train(early_stopping=True, enable_progress_bar=True,accelerator="cuda",devices=1)

adata2.obsm['X_scVI'] = model2.get_latent_representation()


import cellcharter as cc

adata.obsm['X_scVI'] = np.concatenate([adata1.obsm['X_scVI'], adata2.obsm['X_scVI']], axis=1)
%time %memit cc.gr.aggregate_neighbors(adata, n_layers=4, use_rep='X_scVI')
print(adata.obsm['X_cellcharter'].shape)
adata.write_h5ad(path.replace("datasets","results")+"adata_cellcharter.h5ad")
!nvidia-smi

Fri May  2 19:06:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:3B:00.0 Off |                  N/A |
|  0%   27C    P8             22W /  350W |       0MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, 

Epoch 130/400:  32%|██████████████▉                               | 130/400 [11:16<23:24,  5.20s/it, v_num=1, train_loss_step=5.8e+3, train_loss_epoch=6.33e+3]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 6484.739. Signaling Trainer to stop.
peak memory: 18175.65 MiB, increment: 710.99 MiB
CPU times: user 11min 11s, sys: 5.52 s, total: 11min 17s
Wall time: 11min 17s


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/ws6tg/anaconda3/envs/cellcharter/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Epoch 123/400:  31%|█████████████▊                               | 123/400 [19:40<44:18,  9.60s/it, v_num=1, train_loss_step=1.62e+5, train_loss_epoch=1.62e+5]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 172869.484. Signaling Trainer to stop.
peak memory: 26378.57 MiB, increment: 399.98 MiB
CPU times: user 10min 2s, sys: 9min 39s, total: 19min 42s
Wall time: 19min 41s


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 206.82it/s]


peak memory: 26242.91 MiB, increment: 3.42 MiB
CPU times: user 410 ms, sys: 205 ms, total: 616 ms
Wall time: 936 ms
(9752, 100)
Fri May  2 19:38:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:3B:00.0 Off |                  N/A |
| 30%   31C    P8             21W /  350W |     702MiB /  24576MiB |      0%      Default |
|           